In [46]:
fpath = './fwducs/dec37825.txt'

with open(fpath, encoding='utf-8') as f: # context manager
    lines = ''.join( f.readlines() )

In [51]:
lines

'Memorial descritivo de medição e demarcação da Estação Ecológica Corumbá, localizada à margem da Rodovia\nCorumbá/Arcos, nas proximidades do trevo para a COMIG, no lugar denominado Corumbá, cuja gleba e delimitada por um\npolígono irregular. O ponto de partida foi localizado no marco cravado na distância de 1,00 m, à esquerda do canto\nda cerca de divisa entre a Estação Ecológica, a Rodovia Arcos—Corumbá e Rolando Alves de Souza, com as coordenadasabsolutas E-10.000,000 e N-5.000,000; daí, segue confrontando com Rolando Alves de Souza, com o azimute de 298238\'00", na distância de 296,103 m, até atingir o vértice nº 01, de coordenadas E-9.740,175 e N-5.141,866; segue pela cerca de divisa à esquerda, na distância de 1,50 m; segue a ré, na distância de 276,00 m, pelo Córrego Santo Antônio, e sai; segue a ré, na distância de 193,00 m, e cruza a rede telefônica; daí, segue com o azimute de 301°42\'124", na distância de 128,861 m, até atingir o vértice nº 02, de coordenadas E-9.633,126 e N

---

# Regular Expressions

Vamos começar a ver expressões regulares

In [52]:
import re

In [89]:
# comment
re.findall( '[EN]\-?[0-9\.\-,]+,[0-9]{1,3}' , lines )

['E-10.000,000',
 'N-5.000,000',
 'E-9.740,175',
 'N-5.141,866',
 'E-9.633,126',
 'N-5.208,003',
 'E-9.589,200',
 'N-5.238,457',
 'E-9.661,851',
 'N-5.426,092',
 'E-9.634,180',
 'N-5.422,209',
 'E-9.712,095',
 'N5.551,784',
 'E-9.696,339',
 'N-5.553,894',
 'E-9.719,684',
 'N-5.706,781',
 'E-9.170,600',
 'N-5.598,175',
 'E9.668,726',
 'N-5.632,721',
 'E-9.706,017',
 'N-5.696,076',
 'E-9.689,773',
 'N-5.735,972',
 'E-906,089',
 'N-5.804,720',
 'E-9.575,597',
 'N-6.053,718',
 'E-9.572,885',
 'N-6.076,549',
 'E-9.534,847',
 'N6.068,565',
 'E-9.507,959',
 'N-6.230,886',
 'E-9.499,481',
 'N-6.085,920',
 'E-9.444,098',
 'N-5.979,095',
 'E-9.363,137',
 'N-5.901,473',
 'E-367,445',
 'N5.956,872',
 'E-9.382,204',
 'N-6.072,590',
 'E-9.408,169',
 'N-6.197,588',
 'E-9.352,294',
 'N-6.343,442',
 'E-9.267,145',
 'N-6.545,753',
 'E-9.300,187',
 'N-4.462,741',
 'E-9.235,441',
 'N-6.251,967',
 'E-9.200,010',
 'N-6.602,572',
 'E-9.124,414',
 'N-6.679,944',
 'E-8.843,971',
 'N-6.913,206',
 'E-8.934,036',

In [99]:
d = {
    './fwducs/dec37825.txt': '[EN]\-?[0-9\.\-,]+,[0-9]{1,3}'
}

In [100]:
list(d)

['./fwducs/dec37825.txt']

In [97]:
def pegarCoords( nomeArq, mapa ):
    with open(fpath, encoding='utf-8') as f: # context manager
        lines = ''.join( f.readlines() )
        
    expr = mapa[nomeArq]
    casos = re.findall( expr, lines )
    return casos

In [101]:

fileName = './fwducs/dec37825.txt'

pegarCoords(fileName, d)

['E-10.000,000',
 'N-5.000,000',
 'E-9.740,175',
 'N-5.141,866',
 'E-9.633,126',
 'N-5.208,003',
 'E-9.589,200',
 'N-5.238,457',
 'E-9.661,851',
 'N-5.426,092',
 'E-9.634,180',
 'N-5.422,209',
 'E-9.712,095',
 'N5.551,784',
 'E-9.696,339',
 'N-5.553,894',
 'E-9.719,684',
 'N-5.706,781',
 'E-9.170,600',
 'N-5.598,175',
 'E9.668,726',
 'N-5.632,721',
 'E-9.706,017',
 'N-5.696,076',
 'E-9.689,773',
 'N-5.735,972',
 'E-906,089',
 'N-5.804,720',
 'E-9.575,597',
 'N-6.053,718',
 'E-9.572,885',
 'N-6.076,549',
 'E-9.534,847',
 'N6.068,565',
 'E-9.507,959',
 'N-6.230,886',
 'E-9.499,481',
 'N-6.085,920',
 'E-9.444,098',
 'N-5.979,095',
 'E-9.363,137',
 'N-5.901,473',
 'E-367,445',
 'N5.956,872',
 'E-9.382,204',
 'N-6.072,590',
 'E-9.408,169',
 'N-6.197,588',
 'E-9.352,294',
 'N-6.343,442',
 'E-9.267,145',
 'N-6.545,753',
 'E-9.300,187',
 'N-4.462,741',
 'E-9.235,441',
 'N-6.251,967',
 'E-9.200,010',
 'N-6.602,572',
 'E-9.124,414',
 'N-6.679,944',
 'E-8.843,971',
 'N-6.913,206',
 'E-8.934,036',

In [102]:
d = {
    'a': 1,
    'b': 2,
    'c': 3
}

In [104]:
list(d.items())

[('a', 1), ('b', 2), ('c', 3)]

In [ ]:
for k,v in d.items():
    